In [1]:
import pandas as pd

In [16]:
users = pd.read_excel('users.xlsx')

# users

FileNotFoundError: [Errno 2] No such file or directory: 'users.xlsx'

In [ ]:
query = {"email": {"$in": users['Email'].tolist()}}
# query

In [17]:
_users = pd.read_csv('users.csv')
# _users

FileNotFoundError: [Errno 2] No such file or directory: 'users.csv'

In [18]:
# Find all users in 'users' not in '_users' by email
missing_users = users[~users['Email'].isin(_users['email'])]
# missing_users


In [14]:
missing_users.to_excel('missing_users.xlsx', index=False)

In [19]:
# _users.id.values